# Extract data

In [1]:
! pip install kaggle --quiet
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c m5-forecasting-accuracy
! unzip /content/m5-forecasting-accuracy.zip

  0% 0.00/45.8M [00:00<?, ?B/s]
100% 45.8M/45.8M [00:00<00:00, 728MB/s]
Archive:  /content/m5-forecasting-accuracy.zip
  inflating: calendar.csv            
  inflating: sales_train_evaluation.csv  
  inflating: sales_train_validation.csv  
  inflating: sample_submission.csv   
  inflating: sell_prices.csv         


# Accessing Data

## sales data

In [2]:
import pandas as pd

In [37]:
sales_df = pd.read_csv('/content/sales_train_validation.csv')
sales_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [38]:
sales_df = sales_df[(sales_df['cat_id']=='FOODS') & (sales_df['dept_id']=='FOODS_1') & (sales_df['state_id']=='CA')]

In [39]:
sales_df = sales_df[sales_df['item_id'].str.extract(r'FOODS_(\d)_(\d+)')[1].astype(int).between(1,100)]
sales_df['item_id'].value_counts()

,count
item_id,
FOODS_1_001,4
FOODS_1_002,4
FOODS_1_003,4
FOODS_1_004,4
FOODS_1_005,4
...,...
FOODS_1_095,4
FOODS_1_096,4
FOODS_1_097,4


In [40]:
sales_df.drop(['cat_id', 'state_id'], axis=1, inplace=True)
sales_df.head()

,id,item_id,dept_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,3,0,0,1,4,2,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,CA_1,0,1,0,1,1,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,CA_1,0,0,0,0,1,3,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,CA_1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,CA_1,3,9,3,3,0,2,...,3,1,1,2,0,2,2,1,4,1


In [42]:
id_vars = ['id', 'item_id', 'dept_id', 'store_id']
sales_long = pd.melt(sales_df, id_vars=id_vars, var_name='d', value_name='sales')
sales_long.head()

,id,item_id,dept_id,store_id,d,sales
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,d_1,3
1,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,CA_1,d_1,0
2,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,CA_1,d_1,0
3,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,CA_1,d_1,0
4,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,CA_1,d_1,3


In [43]:
sales_long[sales_long['item_id']=='FOODS_1_001']

,id,item_id,dept_id,store_id,d,sales
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,d_1,3
98,FOODS_1_001_CA_2_validation,FOODS_1_001,FOODS_1,CA_2,d_1,2
196,FOODS_1_001_CA_3_validation,FOODS_1_001,FOODS_1,CA_3,d_1,1
294,FOODS_1_001_CA_4_validation,FOODS_1_001,FOODS_1,CA_4,d_1,0
392,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,d_2,0
...,...,...,...,...,...,...
749406,FOODS_1_001_CA_4_validation,FOODS_1_001,FOODS_1,CA_4,d_1912,1
749504,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,d_1913,0
749602,FOODS_1_001_CA_2_validation,FOODS_1_001,FOODS_1,CA_2,d_1913,4
749700,FOODS_1_001_CA_3_validation,FOODS_1_001,FOODS_1,CA_3,d_1913,0


## calender data




In [44]:
calender_df = pd.read_csv('/content/calendar.csv')
calender_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [45]:
sales_date = sales_long.merge(calender_df[['d','date','wm_yr_wk','weekday']], on='d', how='left')
sales_date.head()

,id,item_id,dept_id,store_id,d,sales,date,wm_yr_wk,weekday
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,d_1,3,2011-01-29,11101,Saturday
1,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday
2,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday
3,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday
4,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,CA_1,d_1,3,2011-01-29,11101,Saturday


## price data

In [46]:
sell_price_df = pd.read_csv('/content/sell_prices.csv')

In [47]:
sales_price = sales_date.merge(sell_price_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

In [48]:
sales_price.head()

,id,item_id,dept_id,store_id,d,sales,date,wm_yr_wk,weekday,sell_price
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,d_1,3,2011-01-29,11101,Saturday,2.00
1,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday,7.88
2,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday,2.88
3,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday,NaN
4,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,CA_1,d_1,3,2011-01-29,11101,Saturday,2.94


In [52]:
(sales_price['sell_price'].isnull().sum()/len(sales_price))*100

np.float64(14.107422895974908)

In [55]:
sales_price_null = sales_price[sales_price['sell_price'].isnull()]

In [58]:
sales_price_null.describe()

,sales,wm_yr_wk,sell_price
count,105791.0,105791.000000,0.0
mean,0.0,11181.069940,NaN
std,0.0,71.831336,NaN
min,0.0,11101.000000,NaN
25%,0.0,11121.000000,NaN
50%,0.0,11146.000000,NaN
75%,0.0,11229.000000,NaN
max,0.0,11506.000000,NaN


In [59]:
sales_price.dropna(inplace=True)

In [66]:
sales_price['date'] = pd.to_datetime(sales_price['date'])

## Expiry date

In [71]:
import numpy as np
item_shelf_life = {item: np.random.randint(1, 11) for item in sales_price['item_id'].unique()}
sales_price['shelf_life'] = sales_price['item_id'].map(item_shelf_life)

In [72]:
sales_price['expiry_date'] = sales_price['date'] + pd.to_timedelta(sales_price['shelf_life'], unit='d')

In [73]:
sales_price.head()

,id,item_id,dept_id,store_id,d,sales,date,wm_yr_wk,weekday,sell_price,shelf_life,expiry_date
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,d_1,3,2011-01-29,11101,Saturday,2.00,10,2011-02-08
1,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday,7.88,7,2011-02-05
2,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday,2.88,8,2011-02-06
4,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,CA_1,d_1,3,2011-01-29,11101,Saturday,2.94,7,2011-02-05
5,FOODS_1_006_CA_1_validation,FOODS_1_006,FOODS_1,CA_1,d_1,0,2011-01-29,11101,Saturday,1.97,8,2011-02-06


In [75]:
sales_price.duplicated().sum()

np.int64(0)

In [76]:
sales_price.isnull().sum()

,0
id,0
item_id,0
dept_id,0
store_id,0
d,0
sales,0
date,0
wm_yr_wk,0
weekday,0
sell_price,0


In [77]:
sales_price.to_csv('data_gathering.csv', index=False)